In [2]:
%pip install langchain langchain-community langchain-openai python-dotenv langgraph

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///inv.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM inventory LIMIT 10;")

sqlite
['inventory']


"[(1, 'Adapter', 'AC to USB-A with button', 1, 'w1'), (2, 'Device', 'Asus mini laptop', 1, 'w1'), (3, 'Adapter', 'Govee 12V - 1.5A', 2, 'w1'), (4, 'Adapter', 'Single 5V 1A', 7, 'w1'), (5, 'Adapter', 'AC to USB-C', 1, 'w1'), (6, 'Adapter', '19V - 2.1A', 1, 'w1'), (7, 'Adapter', '5V - 2A', 2, 'w1'), (8, 'Adapter', '12V - 1.5A', 1, 'w1'), (9, 'Adapter', 'Double 5V - 2.1A', 1, 'w1'), (10, 'Adapter', 'Double 5V - 2.4A', 1, 'w1')]"

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0.3)

In [5]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x11a903610>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x11a903610>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x11a903610>),
 QuerySQLCheckerTool(description='Use this tool to double check if your 

In [6]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)

In [11]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, messages_modifier=system_message)

inputs =  {"messages": [("user", "which category has the most items?")]}
for s in agent_executor.stream(inputs, stream_mode="updates"):
    print(s)
    print("---")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kWmx5CUlBytQbLvT2EDJNomu', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 548, 'total_tokens': 560}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_aa87380ac5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-87ea2059-42ba-4d9e-9d05-c0dc341a16ca-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'call_kWmx5CUlBytQbLvT2EDJNomu'}], usage_metadata={'input_tokens': 548, 'output_tokens': 12, 'total_tokens': 560})]}}
---
{'tools': {'messages': [ToolMessage(content='inventory', name='sql_db_list_tables', tool_call_id='call_kWmx5CUlBytQbLvT2EDJNomu')]}}
---
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_brOWkvihpimRIdNGL6iyOKOM', 'function': {'arguments': '{"table_names":"inventory"}', 'name': 'sql_db_schema'}, 'type

Error in LangChainTracer.on_tool_end callback: TracerException("Found chain run at ID f0b0a0ec-f3a0-49b3-a4c8-1fc18bf5f66a, but expected {'tool'} run.")


{'tools': {'messages': [ToolMessage(content='```sql\nSELECT category, COUNT(*) as item_count \nFROM inventory \nGROUP BY category \nORDER BY item_count DESC \nLIMIT 1;\n```', name='sql_db_query_checker', tool_call_id='call_sL4kyBz3ENk4iMF4lL9BLqqu')]}}
---
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3jrOeKX8zK2YWgoJAp1fC7gy', 'function': {'arguments': '{"query":"SELECT category, COUNT(*) as item_count FROM inventory GROUP BY category ORDER BY item_count DESC LIMIT 1;"}', 'name': 'sql_db_query'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 777, 'total_tokens': 813}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d9afa219-e6e9-48e4-8323-d77dfad17c84-0', tool_calls=[{'name': 'sql_db_query', 'args': {'query': 'SELECT category, COUNT(*) as item_count FROM inventory GROUP BY category ORDER BY item_count DESC LIMIT 1